# 导入需要的库

In [1]:
import pandas as pd
import pyecharts

# 读入数据

In [2]:
df = pd.read_csv(r"D:\data\time_series_covid_19_confirmed.csv")

In [3]:
df.head(8)

Province/State       Country/Region      Lat     Long  1/22/20  1/23/20  \
0            NaN          Afghanistan  33.0000  65.0000        0        0   
1            NaN              Albania  41.1533  20.1683        0        0   
2            NaN              Algeria  28.0339   1.6596        0        0   
3            NaN              Andorra  42.5063   1.5218        0        0   
4            NaN               Angola -11.2027  17.8739        0        0   
5            NaN  Antigua and Barbuda  17.0608 -61.7964        0        0   
6            NaN            Argentina -38.4161 -63.6167        0        0   
7            NaN              Armenia  40.0691  45.0382        0        0   

   1/24/20  1/25/20  1/26/20  1/27/20  ...  4/3/20  4/4/20  4/5/20  4/6/20  \
0        0        0        0        0  ...     281     299     349     367   
1        0        0        0        0  ...     304     333     361     377   
2        0        0        0        0  ...    1171    1251    1320    1423   
3        0        0        0        0  ...     439     466     501     525   
4        0        0        0        0  ...       8      10      14      16   
5        0        0        0        0  ...      15      15      15      15   
6        0        0        0        0  ...    1265    1451    1451    1554   
7        0        0        0        0  ...     736     770     822     833   

   4/7/20  4/8/20  4/9/20  4/10/20  4/11/20  4/12/20  
0     423     444     484      521      555      607  
1     383     400     409      416      433      446  
2    1468    1572    1666     1761     1825     1914  
3     545     564     583      601      601      638  
4      17      19      19       19       19       19  
5      19      19      19       19       21       21  
6    1628    1715    1795     1975     1975     2142  
7     853     881     921      937      967     1013  

[8 rows x 86 columns]

# 查看数据

## 提前替换与地图名称不同的国家名

In [4]:
df["Country/Region"] = df["Country/Region"].str.replace("US","United States") #原表中美国的名字与世界地图上的名字不符，提前进行更改

In [5]:
df.shape

(264, 86)

## 检查缺失值

In [6]:
df.isnull().sum() #检查缺失值

Province/State    182
Country/Region      0
Lat                 0
Long                0
1/22/20             0
                 ... 
4/8/20              0
4/9/20              0
4/10/20             0
4/11/20             0
4/12/20             0
Length: 86, dtype: int64

In [7]:
df.isnull().sum().value_counts() #检查缺失值

0      85
182     1
dtype: int64

In [8]:
df.drop(columns="Province/State",inplace=True) #删除缺失值

## 检查重复值

In [9]:
df.duplicated().value_counts() #检查重复值

False    264
dtype: int64

## 按照国家进行分组汇总

In [10]:
df["Country/Region"].value_counts()

China             33
Canada            15
United Kingdom    11
France            11
Australia          8
                  ..
Jamaica            1
Albania            1
Guyana             1
Kenya              1
Israel             1
Name: Country/Region, Length: 185, dtype: int64

In [11]:
gp1 = df.groupby("Country/Region").sum() #有些国家分州、省统计的数据，依据国家进行分组加和，以国家作为最小统计单位
gp1.head()

Lat     Long  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
Country/Region                                                                  
Afghanistan     33.0000  65.0000        0        0        0        0        0   
Albania         41.1533  20.1683        0        0        0        0        0   
Algeria         28.0339   1.6596        0        0        0        0        0   
Andorra         42.5063   1.5218        0        0        0        0        0   
Angola         -11.2027  17.8739        0        0        0        0        0   

                1/27/20  1/28/20  1/29/20  ...  4/3/20  4/4/20  4/5/20  \
Country/Region                             ...                           
Afghanistan           0        0        0  ...     281     299     349   
Albania               0        0        0  ...     304     333     361   
Algeria               0        0        0  ...    1171    1251    1320   
Andorra               0        0        0  ...     439     466     501   
Angola                0        0        0  ...       8      10      14   

                4/6/20  4/7/20  4/8/20  4/9/20  4/10/20  4/11/20  4/12/20  
Country/Region                                                             
Afghanistan        367     423     444     484      521      555      607  
Albania            377     383     400     409      416      433      446  
Algeria           1423    1468    1572    1666     1761     1825     1914  
Andorra            525     545     564     583      601      601      638  
Angola              16      17      19      19       19       19       19  

[5 rows x 84 columns]

In [12]:
gp2 = gp1.iloc[:,2:] #提取每天每个国家的确诊数据
gp2.head(2)

1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  1/27/20  1/28/20  \
Country/Region                                                                  
Afghanistan           0        0        0        0        0        0        0   
Albania               0        0        0        0        0        0        0   

                1/29/20  1/30/20  1/31/20  ...  4/3/20  4/4/20  4/5/20  \
Country/Region                             ...                           
Afghanistan           0        0        0  ...     281     299     349   
Albania               0        0        0  ...     304     333     361   

                4/6/20  4/7/20  4/8/20  4/9/20  4/10/20  4/11/20  4/12/20  
Country/Region                                                             
Afghanistan        367     423     444     484      521      555      607  
Albania            377     383     400     409      416      433      446  

[2 rows x 82 columns]

## 提取确诊人数前十的国家

In [13]:
country = gp2.iloc[:,-1].sort_values(ascending = False)[:10]  #按照累计确诊人数进行降序排序
country

Country/Region
United States     555313
Spain             166831
Italy             156363
France            133670
Germany           127854
United Kingdom     85206
China              83134
Iran               71686
Turkey             56956
Belgium            29647
Name: 4/12/20, dtype: int64

In [14]:
gp3 = gp2[gp2.index.isin(country.index.tolist())] #从gp2中提取确诊人数前十的国家
gp3

1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  1/27/20  1/28/20  \
Country/Region                                                                  
Belgium               0        0        0        0        0        0        0   
China               548      643      920     1406     2075     2877     5509   
France                0        0        2        3        3        3        4   
Germany               0        0        0        0        0        1        4   
Iran                  0        0        0        0        0        0        0   
Italy                 0        0        0        0        0        0        0   
Spain                 0        0        0        0        0        0        0   
Turkey                0        0        0        0        0        0        0   
United Kingdom        0        0        0        0        0        0        0   
United States         1        1        2        2        5        5        5   

                1/29/20  1/30/20  1/31/20  ...  4/3/20  4/4/20  4/5/20  \
Country/Region                             ...                           
Belgium               0        0        0  ...   16770   18431   19691   
China              6087     8141     9802  ...   82511   82543   82602   
France                5        5        5  ...   65202   90848   93773   
Germany               4        4        5  ...   91159   96092  100123   
Iran                  0        0        0  ...   53183   55743   58226   
Italy                 0        0        2  ...  119827  124632  128948   
Spain                 0        0        0  ...  119199  126168  131646   
Turkey                0        0        0  ...   20921   23934   27069   
United Kingdom        0        0        2  ...   38689   42477   48436   
United States         5        5        7  ...  275586  308853  337072   

                4/6/20  4/7/20  4/8/20  4/9/20  4/10/20  4/11/20  4/12/20  
Country/Region                                                             
Belgium          20814   22194   23403   24983    26667    28018    29647  
China            82665   82718   82809   82883    82941    83014    83134  
France           98963  110065  113959  118781   125931   130727   133670  
Germany         103374  107663  113296  118181   122171   124908   127854  
Iran             60500   62589   64586   66220    68192    70029    71686  
Italy           132547  135586  139422  143626   147577   152271   156363  
Spain           136675  141942  148220  153222   158273   163027   166831  
Turkey           30217   34109   38226   42282    47029    52167    56956  
United Kingdom   52279   55949   61474   65872    74605    79874    85206  
United States   366667  396223  429052  461437   496535   526396   555313  

[10 rows x 82 columns]

In [15]:
index = pd.Series(pd.date_range("1/22/20","4/12/20").strftime('%Y/%m/%d')) #修改时间数据类型为字符

In [16]:
index

0     2020/01/22
1     2020/01/23
2     2020/01/24
3     2020/01/25
4     2020/01/26
         ...    
77    2020/04/08
78    2020/04/09
79    2020/04/10
80    2020/04/11
81    2020/04/12
Length: 82, dtype: object

# 绘制动态条形图

In [17]:
from pyecharts import options as opts
from pyecharts.charts import Bar, Timeline

tl = Timeline()
for i in index :
    bar = (
        Bar()
        .add_xaxis(gp3.index.tolist())
        .add_yaxis("国家",gp3.values.T[index[index.values == i].index][0].tolist(), label_opts=opts.LabelOpts(position="right"))
        .reversal_axis()
        .set_global_opts(
            title_opts=opts.TitleOpts("每天疫情确诊数据 (时间: {} )".format(i))
        )
    )
    tl.add_schema(play_interval = 160,is_loop_play = False).add(bar, "{}".format(i)) #设置间隔时间160毫秒，设置不循环播放
# tl.render("每天疫情确诊情况.html")
tl.render_notebook()

# 绘制动态地图

In [18]:
from pyecharts import options as opts
from pyecharts.charts import Map, Timeline
from pyecharts.faker import Faker

tl = Timeline()
for i in index:
    map0 = (
        Map()
        .add("国家", list(zip(gp2.index,gp2.values.T[index[index.values == i].index][0].tolist())), "world")
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="{}疫情确诊数据".format(i)),
            visualmap_opts=opts.VisualMapOpts(min_=1,max_=100000), #为了能在地图上体现更多的信息，最大值选择100000，有五个超10万的国家
        )
    )
    tl.add_schema(play_interval = 160,is_loop_play = False).add(map0, "{}".format(i))
# tl.render("疫情确诊数据.html")
tl.render_notebook()

In [19]:
from pyecharts import options as opts
from pyecharts.charts import Map, Timeline
from pyecharts.faker import Faker

tl = Timeline()
for i in index:
    map0 = (
        Map()
        .add("国家", list(zip(gp2.index,gp2.values.T[index[index.values == i].index][0].tolist())), "world")
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="{}疫情确诊数据".format(i)),
            visualmap_opts=opts.VisualMapOpts(min_=1,max_=560000), #为了能在地图上体现更多的信息，最大值选择100000，有五个超10万的国家
        )
    )
    tl.add_schema(play_interval = 160,is_loop_play = False).add(map0, "{}".format(i))
# tl.render("疫情确诊数据.html")
tl.render_notebook()

In [20]:
index = pd.Series(pd.date_range("1/22/20","4/12/20").strftime('%m/%d/%y')) #修改时间数据类型为字符

In [21]:
index[index.values == index[0]].index #通过值提取索引

Int64Index([0], dtype='int64')

In [22]:
index

0     01/22/20
1     01/23/20
2     01/24/20
3     01/25/20
4     01/26/20
        ...   
77    04/08/20
78    04/09/20
79    04/10/20
80    04/11/20
81    04/12/20
Length: 82, dtype: object

In [23]:
gp3.columns = index  #修改gp3的列名与格式化的时间数据类型一致

In [24]:
from pyecharts import options as opts
from pyecharts.charts import Bar, Timeline

tl = Timeline()
for i in index :
    bar = (
        Bar()
        .add_xaxis(gp3.loc[:,i].sort_values().index.tolist())
        .add_yaxis("国家",gp3.loc[:,i].sort_values().values.tolist(), label_opts=opts.LabelOpts(position="right"))
        .reversal_axis()
        .set_global_opts(
            title_opts=opts.TitleOpts("每天疫情确诊数据 (时间: {} )".format(i))
        )
    )
    tl.add_schema(play_interval = 200,is_loop_play = False).add(bar, "{}".format(i)) #设置间隔时间200毫秒，设置不循环播放
tl.render("每天疫情确诊情况.html")
tl.render_notebook()